In [70]:
import sklearn
import numpy as np
import pandas as pd
import datetime
from pyts.approximation import SymbolicFourierApproximation
from datetime import datetime
from numpy import inf
from pyts.approximation import DiscreteFourierTransform

# Loading data and adding date column

In [3]:
def do_log(array):
    res = np.log(array)
    res[res == -inf] = 0
    return res

In [4]:
epam = pd.read_csv('../data/epam_clean.csv')
epam.drop(axis=1, columns=['Unnamed: 0'], inplace=True)
epam['timestamp'] =  pd.to_datetime(epam.datetime, format='%Y-%m-%d %H:%M:%S')
epam['date'] = epam.timestamp.dt.date

swepam = pd.read_csv('../data/swepam_clean.csv')
swepam.drop(axis=1, columns=['Unnamed: 0'], inplace=True)
swepam['timestamp'] =  pd.to_datetime(swepam.datetime, format='%Y-%m-%d %H:%M:%S')
swepam['date'] = swepam.timestamp.dt.date

mag = pd.read_csv('../data/mag_clean.csv')
mag.drop(axis=1, columns=['Unnamed: 0'], inplace=True)
mag['timestamp'] =  pd.to_datetime(mag.datetime, format='%Y-%m-%d %H:%M:%S')
mag['date'] = mag.timestamp.dt.date

sis = pd.read_csv('../data/sis_clean.csv')
sis.drop(axis=1, columns=['Unnamed: 0'], inplace=True)
sis['timestamp'] =  pd.to_datetime(sis.datetime, format='%Y-%m-%d %H:%M:%S')
sis['date'] = sis.timestamp.dt.date

In [5]:
epam.describe()

,38-53,175-315,47-65,112-187,310-580,761-1220,060-1910,anis_ratio
count,1.962946e+06,1.962946e+06,1.962946e+06,1.962946e+06,1.962946e+06,1.962946e+06,1.962946e+06,1.962946e+06
mean,1.520371e+04,6.261688e+02,1.361767e+04,2.815987e+03,3.192554e+02,1.138035e+02,4.525672e+01,-2.984247e-01
std,2.140501e+05,1.530765e+04,1.748178e+05,4.626019e+04,8.497807e+03,2.364054e+03,9.400059e+02,7.290152e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00
25%,5.100000e+02,2.080000e+01,1.040000e+03,1.880000e+01,2.310000e+00,5.720000e-01,1.470000e-01,-1.000000e+00
50%,7.660000e+02,2.660000e+01,1.640000e+03,2.340000e+01,2.970000e+00,7.810000e-01,2.170000e-01,2.000000e-02
75%,1.330000e+03,3.210000e+01,2.870000e+03,7.910000e+01,9.920000e+00,2.680000e+00,8.020000e-01,3.200000e-01
max,5.790000e+07,4.460000e+06,2.070000e+07,8.730000e+06,6.050000e+06,1.600000e+06,5.140000e+05,2.000000e+00


In [6]:
epam_wanted_columns = ['38-53', '175-315', '47-65', '112-187', '310-580',
                       '761-1220', '060-1910', 'anis_ratio', 'date']
epam[epam_wanted_columns[:-2]] = epam[epam_wanted_columns[:-2]].apply(lambda x: do_log(x))
epam.describe()

/opt/homebrew/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,38-53,175-315,47-65,112-187,310-580,761-1220,060-1910,anis_ratio
count,1.962946e+06,1.962946e+06,1.962946e+06,1.962946e+06,1.962946e+06,1.962946e+06,1.962946e+06,1.962946e+06
mean,6.759110e+00,3.142484e+00,7.408867e+00,3.596544e+00,1.490704e+00,1.471185e-01,-1.127796e+00,-2.984247e-01
std,4.358333e+00,5.271937e+00,2.914607e+00,5.558492e+00,6.042662e+00,6.201975e+00,6.269949e+00,7.290152e-01
min,-1.002841e+02,-1.014869e+02,-9.828361e+01,-1.014869e+02,-1.018918e+02,-1.032799e+02,-1.032799e+02,-1.000000e+00
25%,6.234411e+00,3.034953e+00,6.946976e+00,2.933857e+00,8.372475e-01,-5.586163e-01,-1.910543e+00,-1.000000e+00
50%,6.641182e+00,3.280911e+00,7.402452e+00,3.152736e+00,1.088562e+00,-2.471801e-01,-1.509593e+00,2.000000e-02
75%,7.192934e+00,3.468856e+00,7.962067e+00,4.370713e+00,2.294553e+00,9.858168e-01,-1.142891e-01,3.200000e-01
max,1.787423e+01,1.531066e+01,1.684564e+01,1.598228e+01,1.561557e+01,1.428551e+01,1.314998e+01,2.000000e+00


In [7]:
swepam.describe()

,proton_density,bulk_speed,ion_temperature
count,8.506059e+06,8.506059e+06,8.506059e+06
mean,3.643107e+00,4.276234e+02,9.294717e+04
std,3.909584e+00,1.008328e+02,8.480196e+04
min,0.000000e+00,2.284000e+02,1.000000e+04
25%,1.500000e+00,3.548000e+02,3.770000e+04
50%,2.700000e+00,4.042000e+02,6.670000e+04
75%,4.600000e+00,4.768000e+02,1.200000e+05
max,1.998000e+02,1.341300e+03,5.020000e+06


In [8]:
swepam_wanted_columns = ['proton_density', 'bulk_speed', 'ion_temperature', 'date']
swepam[swepam_wanted_columns[:-1]] = swepam[swepam_wanted_columns[:-1]].apply(lambda x: do_log(x))
swepam.describe()

/opt/homebrew/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,proton_density,bulk_speed,ion_temperature
count,8.506059e+06,8.506059e+06,8.506059e+06
mean,9.172140e-01,6.032865e+00,1.111388e+01
std,9.262688e-01,2.211501e-01,8.114012e-01
min,-2.302585e+00,5.431098e+00,9.210340e+00
25%,4.054651e-01,5.871554e+00,1.053742e+01
50%,9.932518e-01,6.001910e+00,1.110796e+01
75%,1.526056e+00,6.167097e+00,1.169525e+01
max,5.297317e+00,7.201395e+00,1.542894e+01


In [20]:
mag_wanted_columns = ['Bx', 'By', 'Bz', 'Bt', 'Lat.', 'Long.', 'date']
mag.describe()

,Bx,By,Bz,Bt,Lat.,Long.
count,9.993374e+06,9.993374e+06,9.993374e+06,9.993374e+06,9.993374e+06,9.993374e+06
mean,-4.935891e-01,-7.317415e-02,1.667067e-02,5.490971e+00,3.053756e-01,1.979020e+02
std,3.623578e+00,3.831787e+00,3.377525e+00,3.052438e+00,3.353761e+01,9.809418e+01
min,-4.540000e+01,-7.430000e+01,-6.900000e+01,0.000000e+00,-8.990000e+01,0.000000e+00
25%,-3.100000e+00,-2.500000e+00,-1.800000e+00,3.600000e+00,-2.300000e+01,1.260000e+02
50%,-4.000000e-01,-1.000000e-01,0.000000e+00,4.800000e+00,2.000000e-01,1.843000e+02
75%,2.100000e+00,2.300000e+00,1.800000e+00,6.600000e+00,2.360000e+01,2.911000e+02
max,1.232000e+02,5.840000e+01,6.290000e+01,1.394000e+02,9.000000e+01,3.600000e+02


In [10]:
sis.describe()

,> 10 MeV,> 30 MeV
count,2.009789e+06,2.009789e+06
mean,2.578156e+01,1.002080e+01
std,4.129353e+02,2.017720e+02
min,1.890000e-01,1.100000e-01
25%,1.230000e+00,8.380000e-01
50%,1.800000e+00,1.200000e+00
75%,2.260000e+00,1.550000e+00
max,1.230000e+04,7.490000e+03


In [11]:
sis_wanted_columns = ['> 10 MeV', '> 30 MeV', 'date']
sis[sis_wanted_columns[:-1]] = sis[sis_wanted_columns[:-1]].apply(lambda x: do_log(x))
sis.describe()

,> 10 MeV,> 30 MeV
count,2.009789e+06,2.009789e+06
mean,6.183918e-01,1.709641e-01
std,8.330049e-01,6.454362e-01
min,-1.666008e+00,-2.207275e+00
25%,2.070142e-01,-1.767372e-01
50%,5.877867e-01,1.823216e-01
75%,8.153648e-01,4.382549e-01
max,9.417355e+00,8.921324e+00


# Aggregating by date using SFA

### Putting all column values in one list per day

In [35]:
resM = pd.concat([sis[sis_wanted_columns[:-1]], epam[epam_wanted_columns]], axis=1, join="inner")
resV = pd.concat([mag[mag_wanted_columns[:-1]], swepam[swepam_wanted_columns]], axis=1, join="inner")

In [38]:
resM.corr()
# > 10 MeV, 38-53, 175-315, 112-187, 310-580, 761-1220

,> 10 MeV,> 30 MeV,38-53,175-315,47-65,112-187,310-580,761-1220,060-1910,anis_ratio
> 10 MeV,1.000000,0.910580,0.027155,0.031804,0.075242,0.007712,0.003840,0.004635,0.001673,-0.060165
> 30 MeV,0.910580,1.000000,-0.005647,0.010412,0.057880,-0.031837,-0.036363,-0.036324,-0.041500,-0.096304
38-53,0.027155,-0.005647,1.000000,0.735484,0.567398,0.681313,0.636100,0.638236,0.635060,0.005551
175-315,0.031804,0.010412,0.735484,1.000000,0.491903,0.838428,0.785628,0.776741,0.769600,0.062072
47-65,0.075242,0.057880,0.567398,0.491903,1.000000,0.492427,0.445764,0.430011,0.421584,-0.092288
112-187,0.007712,-0.031837,0.681313,0.838428,0.492427,1.000000,0.897358,0.873768,0.861776,0.054574
310-580,0.003840,-0.036363,0.636100,0.785628,0.445764,0.897358,1.000000,0.957298,0.949993,0.055751
761-1220,0.004635,-0.036324,0.638236,0.776741,0.430011,0.873768,0.957298,1.000000,0.969037,0.049401
060-1910,0.001673,-0.041500,0.635060,0.769600,0.421584,0.861776,0.949993,0.969037,1.000000,0.053898
anis_ratio,-0.060165,-0.096304,0.005551,0.062072,-0.092288,0.054574,0.055751,0.049401,0.053898,1.000000


In [41]:
resM.drop(["> 10 MeV", "38-53", "175-315", "112-187", "310-580", "761-1220"], axis=1, inplace=True)
resM.head(1)

,> 30 MeV,47-65,060-1910,anis_ratio,date
0,-0.53273,7.029973,-1.645065,0.9,2001-08-07


In [42]:
resV.corr()
# By, Bz, bulk_speed

,Bx,By,Bz,Bt,Lat.,Long.,proton_density,bulk_speed,ion_temperature
Bx,1.000000,-0.352516,0.090101,-0.131396,0.069871,0.302047,-0.012098,-0.025890,-0.024747
By,-0.352516,1.000000,-0.052711,0.049743,-0.042389,-0.661474,0.015950,0.010201,0.011330
Bz,0.090101,-0.052711,1.000000,-0.022284,0.861256,0.040901,-0.000249,-0.000530,-0.001794
Bt,-0.131396,0.049743,-0.022284,1.000000,-0.015066,-0.023056,0.054094,0.054849,0.054141
Lat.,0.069871,-0.042389,0.861256,-0.015066,1.000000,0.038761,-0.006132,-0.002086,-0.004243
Long.,0.302047,-0.661474,0.040901,-0.023056,0.038761,1.000000,-0.010195,-0.004715,-0.006333
proton_density,-0.012098,0.015950,-0.000249,0.054094,-0.006132,-0.010195,1.000000,-0.182728,-0.114948
bulk_speed,-0.025890,0.010201,-0.000530,0.054849,-0.002086,-0.004715,-0.182728,1.000000,0.770499
ion_temperature,-0.024747,0.011330,-0.001794,0.054141,-0.004243,-0.006333,-0.114948,0.770499,1.000000


In [43]:
resV.drop(["By", "Bz", "bulk_speed"], axis=1, inplace=True)
resV.head(1)

,Bx,Bt,Lat.,Long.,proton_density,ion_temperature,date
0,-6.2,6.9,-0.6,206.5,1.163151,11.138959,2001-08-07


In [44]:
resM_agg_grouped = resM.groupby(['date'])
resM_listed = resM_agg_grouped.agg(lambda x : list(x.to_numpy()))
resM_listed.set_index(pd.DatetimeIndex(resM_listed.index), inplace=True)
resM_listed.head(1)

,> 30 MeV,47-65,060-1910,anis_ratio
date,,,,
2001-08-07,"[-0.5327304591540407, -0.5551258826625707, -0....","[7.029972911706386, 7.098375638590786, 7.35244...","[-1.6450650900772514, -1.3783261914707137, -1....","[0.9, 0.68, 1.38, 0.89, 0.43, 0.83, 0.94, 1.02..."


In [45]:
resV_agg_grouped = resV.groupby(['date'])
resV_listed = resV_agg_grouped.agg(lambda x : list(x.to_numpy()))
resV_listed.set_index(pd.DatetimeIndex(resV_listed.index), inplace=True)
resV_listed.head(1)

,Bx,Bt,Lat.,Long.,proton_density,ion_temperature
date,,,,,,
2001-08-07,"[-6.2, -6.3, -6.2, -6.1, -6.1, -6.1, -6.5, -6....","[6.9, 7.1, 6.9, 7.0, 7.0, 7.0, 7.2, 7.2, 7.0, ...","[-0.6, -2.7, -4.7, -4.8, -3.1, 0.2, -2.3, -0.4...","[206.5, 207.8, 206.5, 209.2, 210.0, 209.8, 204...","[1.1631508098056809, 1.1631508098056809, 1.163...","[11.138959023921435, 11.071314910225711, 11.17..."


# SFA

SFA demands that timeseries that are being transformed have the same number of timestamps.

SFA is done on two df because (sis, epam) are on 5min basis and (mag, swepam) are on a minute basis so they have different number of timestamps per day.

### Adjust length of daily timeseries

If timeseries happen to vary a bit in length due to some previous error, function slick_array will reduce their length to the length of the shortest

In [46]:
def slick_array(array):
    listic = list(array)
    min_size = min(list(map(len, listic)))
    return np.array(list(map(lambda x: x[:min_size], listic)))

### Making an object required for transformation
Each field in dataframe contains a list that represents a daily timeseries. Each list in one row needs to be turned to numpy.array and all of lists in one row need to be put in one numpy.array. For some reason I couldn't manage to do a more simple job :)

In [47]:
def list_to_numpy(listici_arg):
    listici = slick_array(listici_arg)
    res = np.array([listici[0]]).reshape(-1, 1)
    for listic in listici[1:]:
        res = np.concatenate((res, np.array([listic]).reshape(-1, 1)), axis = 1)
    return res

In [48]:
def transform(array, num_coefs=2, num_bins=4):
    x = list_to_numpy(array)
    if x.shape[1] < num_bins:
        return []
    if x.shape[0] <= num_coefs:
        return []
    transformer = SymbolicFourierApproximation(n_coefs=num_coefs, n_bins=num_bins)
    transformed = transformer.fit_transform(np.transpose(x))
    return transformed.flatten()

# Adding class label

### Loading flood dates

In [55]:
floods = pd.read_csv("../data/floods.csv", index_col="date").drop(["Unnamed: 0"], axis =1)
floods.set_index(pd.DatetimeIndex(floods.index), inplace=True)
floods_new = pd.read_csv("../data/floodphenomena.csv")
floods_new = floods_new[(floods_new["StartDate"].notna()) | (floods_new["EndDate"].notna())]
floods_new["StartDateTime"] = pd.to_datetime(floods_new["StartDate"], infer_datetime_format=True)
floods_new["StartDate"] = floods_new["StartDateTime"].dt.date
floods_new["EndDateTime"] = pd.to_datetime(floods_new["EndDate"], infer_datetime_format=True)
floods_new["EndDate"] = floods_new["EndDateTime"].dt.date
floods_new = floods_new[floods_new["StartDate"] >= datetime.strptime("2001-01-01", "%Y-%m-%d").date()]
flood_dates = pd.concat([floods_new["StartDateTime"], floods[floods["flood"]].index.to_series()], axis = 0)

### Adding class according to flood dates
True  -> date from resulting aggregation is in flood dates

False -> otherwise

Also, join lists into one string

In [64]:
resV_listed.columns

Index(['Bx', 'Bt', 'Lat.', 'Long.', 'proton_density', 'ion_temperature'], dtype='object')

### SFA transformation

In [53]:
def flatten_row(row):
    return np.concatenate((row[0], row[1]))

In [69]:
for n_coefs in range(2, 6):
    for n_bins in range(2, 6):
        resM_transformed = resM_listed.apply(transform, axis=1, args=(n_coefs, n_bins))
        resV_transformed = resV_listed.apply(transform, axis=1, args=(n_coefs, n_bins))
        res = pd.concat([resM_transformed, resV_transformed], axis=1, join="inner").apply(flatten_row, axis=1)
        df = pd.DataFrame(columns=["data", "class"], index=res.index)
        df["data"] = res.apply(lambda x: "".join(x))
        df["class"] = res.index.isin(flood_dates)
        print(n_coefs, n_bins)
        df.to_csv("../data/input_log/sfa_" + str(n_coefs)+"_" + str(n_bins)+".csv")

3 4
3 5
4 4
4 5
5 4
5 5
